# Apparel Colorization

Copy an image with color variations

In [1]:
import os
import cv2
import csv
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# check cuda gpu
print(cv2.cuda.getCudaEnabledDeviceCount())

0


In [3]:
# -- RESOURCES
#BASE_DIR = os.path.dirname(__file__)
BASE_DIR = os.getcwd()
TEMPLATE_IMG = os.path.join(BASE_DIR, 'assets', 'template.jpg')
OUTPUT_DIR = os.path.join(BASE_DIR, 'outpu')
COLORS = os.path.join(BASE_DIR, 'data', 'color_combinations.csv')
BODY_MASK = os.path.join(BASE_DIR, 'assets', 'body_mask.png')
POCKETS_MASK = os.path.join(BASE_DIR, 'assets', 'pockets_mask.png')
WEBBINGS_MASK = os.path.join(BASE_DIR, 'assets', 'webbings_mask.png')
os.makedirs(OUTPUT_DIR, exist_ok=True)

### Pseudocode / Workflow
- [x] Define global variables
- [x] Helper functions
- [x] Load assets
- [x] Binarize masks
- [x] Convert template to LAB
- [x] Display images to validation
- [ ] Loop through color combinations
- [ ] Show color to apply
- [ ] Apply masks
- [ ] Show each colorized image as RGB
- [ ] Save images

## HELPER FUNCTIONS

In [4]:
# --- hex color to RGB
def hex_to_bgr(hex_color:str) -> tuple[int, int, int]:
    """Converts a #RRGGBB string to RGB tuple."""
    hex_color = hex_color.strip().lstrip('#')
    if len(hex_color) != 6:
        raise ValueError(f"Invalid hex colour: {hex_color}")
    r:int = int(hex_color[0:2], 16)
    g:int = int(hex_color[2:4], 16) 
    b:int = int(hex_color[4:6], 16)
    return (b,g,r)

In [5]:
def load_mask(path: str, shape: tuple[int, int])-> np.ndarray:
    """Loads mask, binarize, resize if required, returns a uint8 binary array"""
    mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        raise FileNotFoundError(path)
    if mask.shape[:2] != shape:
        mask = cv2.resize(mask, (shape[1], shape[2]), interpolation=cv2.INTER_NEAREST)
    # binarize
    _, mask = cv2.threshold(mask, 127, 1, cv2.THRESH_BINARY)
    return mask.astype(np.uint8)

## IMAGE LOAD AND PREPROCESSING

In [6]:
# load assets: images are loaded as BGR by default on opencv *****
template_raw = cv2.imread(TEMPLATE_IMG)
if template_raw is None:
    raise FileNotFoundError(TEMPLATE_IMG)
H, W = template_raw.shape[:2]

In [7]:
# preprocess masks: resize and convert to binaries
body_mask = load_mask(BODY_MASK, (H,W))
pockets_mask = load_mask(POCKETS_MASK, (H,W))
webbings_mask = load_mask(WEBBINGS_MASK, (H,W))

In [14]:
# ///////////////// combinar masks
body_mask_inds = body_mask.astype(bool)
pockets_mask_inds = pockets_mask.astype(bool)

In [15]:
# convert template to LAB to preserve luminosity, extract layers
template_lab = cv2.cvtColor(template_raw, cv2.COLOR_BGR2LAB)
L_template, A_template, B_template = cv2.split(template_lab)

In [26]:
# convert template to HSV 
template_hsb = cv2.cvtColor(template_raw, cv2.COLOR_BGR2HSV)
H_template, S_template, V_template = cv2.split(template_hsb)

In [16]:
# for visualization, transform loaded images to RGB, opencv transformed to BGR.
template_raw_rgb = cv2.cvtColor(template_raw, cv2.COLOR_BGR2RGB)

In [ ]:
images = [template_raw_rgb, template_raw, template_lab, L_template, A_template, B_template, H_template, S_template, V_template, body_mask, pockets_mask, webbings_mask, body_mask_inds, pockets_mask_inds]
titles = ["Template", "Template BGR", "Template LAB", "Template L", "Template A", "Template B", "Template H", "Template S", "Template V","Body Mask", "Pockets Mask", "Webbings Mask", "Body-Mask_INDS", "Pocket-Mask_INDS"]
cols=3
rows=5
fig, axes = plt.subplots(rows, cols, figsize=(10,8))

for i, ax in enumerate(axes.flatten()):
    if i < len(images):
        ax.imshow(images[i])
        ax.set_title(f"{titles[i]}")
        ax.axis('off')

plt.tight_layout
plt.show()



# COLOR PROCESSING

In [28]:
# load colorization list
with open(COLORS, newline='') as colors:
    reader = csv.DictReader(colors)
    combos = list(reader)

print(f"Generating {len(combos)} colors ...")

Generating 16 colors ...


In [29]:
# loop colors
for row in combos:
    file_base = os.path.splitext(row['filename'].strip())[0]
    body_hex = row['body_color'].strip()
    pockets_hex = row['pockets_color'].strip()

    # convert to BGR 
    body_bgr = np.array([[hex_to_bgr(body_hex)]], dtype=np.uint8)
    pockets_bgr = np.array([[hex_to_bgr(pockets_hex)]], dtype=np.uint8)
    
    print(f"BGR\nfile: {file_base} | body_color: {body_bgr} | pockets: {pockets_bgr}")
    
    # convert to LAB
    body_lab = cv2.cvtColor(body_bgr, cv2.COLOR_BGR2LAB)[0,0]
    pockets_lab = cv2.cvtColor(pockets_bgr, cv2.COLOR_BGR2LAB)[0,0]

    print(f"LAB\nfile: {file_base} | body_color: {body_lab} | pockets: {pockets_lab}")

    # Clone original LAB channels
    L = L_template.copy()
    A = A_template.copy()
    B = B_template.copy()

    # Change body region hue,sat
    A[body_mask_inds] = body_lab[1]
    B[body_mask_inds] = body_lab[2]
    # Change pockets region hue,sat
    A[pockets_mask_inds] = pockets_lab[1]
    B[pockets_mask_inds] = pockets_lab[2]

    # hard clamp ....

    # merge and convert to BGR
    lab_colored = cv2.merge([L, A, B])
    out_bgr = cv2.cvtColor(lab_colored, cv2.COLOR_LAB2BGR)
    out_bgr = cv2.cvtColor(out_bgr, cv2.COLOR_BGR2RGB)

    # Save
    out_jpg = os.path.join(OUTPUT_DIR, f"{file_base}_v5.jpg")
    cv2.imwrite(out_jpg, out_bgr)
    

BGR
file: delantal_gairaca | body_color: [[[0 0 0]]] | pockets: [[[ 33  25 179]]]
LAB
file: delantal_gairaca | body_color: [  0 128 128] | pockets: [ 98 187 166]
BGR
file: delantal_santa_marta | body_color: [[[64 45 38]]] | pockets: [[[ 38  96 252]]]
LAB
file: delantal_santa_marta | body_color: [ 48 131 115] | pockets: [156 185 189]
BGR
file: delantal_mendihuaca | body_color: [[[101 114 130]]] | pockets: [[[ 23   1 203]]]
LAB
file: delantal_mendihuaca | body_color: [125 132 137] | pockets: [108 195 176]
BGR
file: delantal_waikiki | body_color: [[[198 206 212]]] | pockets: [[[64 45 38]]]
LAB
file: delantal_waikiki | body_color: [212 129 133] | pockets: [ 48 131 115]
BGR
file: delantal_carey | body_color: [[[ 38  96 252]]] | pockets: [[[198 206 212]]]
LAB
file: delantal_carey | body_color: [156 185 189] | pockets: [212 129 133]
BGR
file: delantal_arrecife | body_color: [[[64 45 38]]] | pockets: [[[ 64 204 238]]]
LAB
file: delantal_arrecife | body_color: [ 48 131 115] | pockets: [211 126 